# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load output.csv file to a DataFrame
file_path="../output_data/cities.csv"
data=pd.read_csv(file_path, encoding="utf-8")
data.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ilo,-17.6394,-71.3375,71.15,62,2,5.82,PE,1587748514
1,Souillac,-20.5167,57.5167,77.00,94,75,8.05,MU,1587747928
2,Hasaki,35.7333,140.8333,41.00,100,20,4.70,JP,1587748159
3,Rikitea,-23.1203,-134.9692,76.55,70,100,11.14,PF,1587748521
4,Yarim,14.2980,44.3779,57.18,42,97,1.32,YE,1587748142


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longtitude as locations
city_locations=data[["Latitude","Longitude"]]

# Drop NaN values from Humidity values and set the data type as float
humidity=data["Humidity"].dropna().astype("float")
max_humidity=humidity.max()

In [5]:
# Customize the figure layout
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [6]:
# Display the gmap
fig = gmaps.figure(layout=figure_layout,zoom_level=2, center=(0,0))
# Assign a heatmap layer to a variable
heatmap_layer = gmaps.heatmap_layer(city_locations, weights=humidity,dissipating=False, max_intensity=max_humidity, point_radius=3)
# Add a heatmap layer to the map
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Filter rows with ideal weather conditions
ideal_weather_df=data.loc[(data["Max Temp"]<80) & (data["Max Temp"]>70) & \
                         (data["Wind Speed"]<10) & (data["Cloudiness"]==0),:]
# Drop NaN from the dataframe
ideal_weather_df = ideal_weather_df.dropna(how='any')
# Reset index
ideal_weather_df = ideal_weather_df.reset_index(drop=True)
ideal_weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nālūt,30.3333,10.8500,77.22,15,0,7.76,LY,1587748273
1,Telêmaco Borba,-24.3239,-50.6156,78.62,39,0,5.26,BR,1587748480
2,San Quintín,30.4833,-115.9500,73.83,45,0,5.99,MX,1587748514
3,Empangeni,-28.7620,31.8933,71.78,79,0,9.78,ZA,1587748148
4,Pisco,-13.7000,-76.2167,73.40,73,0,9.17,PE,1587748514
5,Tonneins,44.3921,0.3124,77.00,50,0,2.24,FR,1587748390


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Set up a new dataframe with four columns
hotel_df= ideal_weather_df[["City","Country","Latitude","Longitude"]]
# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-8-f5c71e4ca14f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitude,Hotel Name
0,Nālūt,LY,30.3333,10.8500,
1,Telêmaco Borba,BR,-24.3239,-50.6156,
2,San Quintín,MX,30.4833,-115.9500,
3,Empangeni,ZA,-28.7620,31.8933,
4,Pisco,PE,-13.7000,-76.2167,
5,Tonneins,FR,44.3921,0.3124,


In [9]:
nearby_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={
    "key":g_key,
    "radius":5000,
    "keyword": "hotel",
    "types":"lodging"
}

# Loop through each row in the dataframe
for index,row in hotel_df.iterrows():
    # Add a "location" key to the params dictionary
    params["location"]=f"{row['Latitude']},{row['Longitude']}" 
    response_json=requests.get(nearby_url,params=params).json()
    
    try:
        # Assign the name of the hotel to the "Hotel Name" column
        hotel_df.loc[index,"Hotel Name"]=response_json['results'][0]['name']
    except:
        # If the hotel isn't found, assign NaN instead
        hotel_df.loc[index,"Hotel Name"]=np.nan

/Users/juahan/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Nālūt,LY,30.3333,10.8500,NaN
1,Telêmaco Borba,BR,-24.3239,-50.6156,DOM SOLLIEVO HOTEL
2,San Quintín,MX,30.4833,-115.9500,Hotel Jardines Baja
3,Empangeni,ZA,-28.7620,31.8933,BON Hotel Empangeni
4,Pisco,PE,-13.7000,-76.2167,Embassy Beach
5,Tonneins,FR,44.3921,0.3124,Château Marith


In [11]:
#Drop NaN from the dataframe
hotel_df = hotel_df.dropna(how='any')
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,Telêmaco Borba,BR,-24.3239,-50.6156,DOM SOLLIEVO HOTEL
2,San Quintín,MX,30.4833,-115.9500,Hotel Jardines Baja
3,Empangeni,ZA,-28.7620,31.8933,BON Hotel Empangeni
4,Pisco,PE,-13.7000,-76.2167,Embassy Beach
5,Tonneins,FR,44.3921,0.3124,Château Marith


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…